In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

import os
import cv2
import random as rnd

In [ ]:
DATADIR = 'C:\\Users\\xboxd\\OneDrive\\Projects\\ImageClassification\\data'
CATEGORIES = ['real', 'fake']
IMAGE_SIZE = 100

In [ ]:
train_data = []

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for image in os.listdir(path):
        try:
            image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
            new_image_array = cv2.resize(image_array, (IMAGE_SIZE, IMAGE_SIZE))
            train_data.append([new_image_array, class_num])
        except Exception as e:
            pass

rnd.shuffle(train_data)

In [ ]:
r = rnd.randint(0, len(train_data))
plt.imshow(train_data[r][0], cmap='gray')
plt.show
print(CATEGORIES[train_data[r][1]])

In [ ]:
X = []
y = []

for feature, label in train_data:
    X.append(feature)
    y.append(label)

X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train = keras.utils.normalize(X_train, axis=1)
X_test = keras.utils.normalize(X_test, axis=1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.MaxPool2D(pool_size=(2, 2)),

    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Nadam(learning_rate=1e-3),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
plt.plot(history.epoch, history.history['accuracy'])
plt.show()

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=False)
print('loss = ', loss)
print('accuracy = ', acc)